In [1]:
import concurrent.futures
import time

start = time.perf_counter()


def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    results = executor.map(do_something, secs)

    # for result in results:
    #     print(result)

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 0.4 second(s)


## running tasks synchronously vs. distributing these tasks across multiple CPUs

task is either I/O bound or CPU bound

I/O file system ops, network ops

threading on CPU bound tasks - running only one process

multiple processing - for both kinds of tasks

run something in parallel with multiprocessing

## POOLS allows us to add multiprocessing to our program with just a few lines of code

In [2]:
import multiprocessing
# import concurrent.futures
# import time

start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

# create 2 different processes

p1 = multiprocessing.Process(target=do_something)
p2 = multiprocessing.Process(target=do_something)

# in order for our functions to run, we have to use the start method

p1.start()
p2.start()

# for the processes to run and finish before moving on to the rest of the script, use 
# join method

p1.join()
p2.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')



Finished in 0.32 second(s)


In [3]:
# now let's create and start these multiple processes in a loop
from tqdm import tqdm
start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

processes=[]

for _ in tqdm(range(10)): # _ is just used as a throwaway variable since we're not using those integers
    # anywhere in the loop
    p = multiprocessing.Process(target=do_something, args=[1.5])
    # now unlike with threads, in order to pass arguments to a multiprocessing process
    # the arguments must be able to be serialized using PICKLE
    # PICKLE - converting python objects to a format that can be deconstructed and 
    # reconstructed in another python script, so we expect our function to take 1.5 secs as
    # an argument 
    p.start()
    # we can't use .join() method in the loop because that'd make it synchronous
    processes.append(p)
    
for process in processes:
    process.join()
    

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

    

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 90.08it/s]


Finished in 0.47 second(s)


## Faster Easier Way of Doing Multiprocessing

## Process Pool Executor - allows us to switch to multiple threads / processes



In [4]:
import concurrent.futures

# import concurrent.futures
# import time

start = time.perf_counter()

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

# use a context manager
with concurrent.futures.ProcessPoolExecutor() as executor:
    #f1 = executor.submit(do_something,1)
    # above line submits the function to be executed and returns a future object
    # f1 encapsulates the future execution of our function
    #print(f1.result())
    #secs = [5, 4, 3, 2, 1]
    #results=[executor.submit(do_something,sec) for sec in secs]
#     for f in concurrent.futures.as_completed(results):
#         print(f.result())
    secs = [5, 4, 3, 2, 1]
    results = executor.map(do_something, secs)

#     for result in results:
#         print(result)

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')



Finished in 0.4 second(s)


## CLEAR-CUT DIFFERENCE BETWEEN THREADING AND MULTIPROCESSING IN PYTHON

In [5]:
# Threading

from threading import Thread
import os
import math
from tqdm import tqdm

start = time.perf_counter()

def calc():
	for i in range(0, 4000000):
		math.sqrt(i)

threads = []

for i in tqdm(range(os.cpu_count())):
	print('registering thread %d' % i)
	threads.append(Thread(target=calc))

for thread in tqdm(threads):
	thread.start()

for thread in tqdm(threads):
	thread.join()
    
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

 38%|███████████████████████████████▌                                                    | 3/8 [00:00<00:00, 23.51it/s]

registering thread 0
registering thread 1
registering thread 2
registering thread 3
registering thread 4
registering thread 5
registering thread 6
registering thread 7


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.10s/it]

Finished in 12.93 second(s)


In [6]:
# Multiprocessing

from multiprocessing import Process
# import os
# import math

start = time.perf_counter()

def calc():
	for i in range(0, 70000000):
		math.sqrt(i)

processes = []

for i in tqdm(range(os.cpu_count())):
	print('registering process %d' % i)
	processes.append(Process(target=calc))

for process in tqdm(processes):
	process.start()

for process in tqdm(processes):
	process.join()
    
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

registering process 0
registering process 1
registering process 2
registering process 3
registering process 4
registering process 5
registering process 6
registering process 7


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 21.99it/s]

Finished in 0.49 second(s)
